In [ ]:
!git clone https://github.com/FedericoArcelaschi-Polimi/edited_RecSyS_Course_AT_PoliMi

In [ ]:
cd edited_RecSyS_Course_AT_PoliMi

In [ ]:
!python run_compile_all_cython.py

In [ ]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline
%load_ext Cython

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_SVDpp_Cython
import optuna

In [ ]:
data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
# data_train_path="data_train.csv"
# data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [ ]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

In [ ]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
def train_evaluate(optuna_trial):
    factors = optuna_trial.suggest_int("factors", 1, 1000)
    
    mAP = 0.0
    for i in range(0, test_folds):
        recommender = MatrixFactorization_SVDpp_Cython(URM_trains[i], verbose=False)
        recommender.fit(num_factors=factors)
        result_df, _ = evaluator_tests[i].evaluateRecommender(recommender)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [ ]:
storage = optuna.storages.JournalStorage(
    optuna.storages.JournalFileStorage("studies/SVDpp_study.log"),  # NFS path for distributed optimization
)
study = optuna.load_study(study_name = "SVDpp", storage = storage)
study.optimize(train_evaluate, n_trials=600)

In [ ]:
!cp studies/SVDpp_study.log ../SVDpp_study.log

In [ ]:
import time

topk = study.best_params["topK"]
shrink = study.best_params["shrink"]

recommender = MatrixFactorization_SVDpp_Cython(URM_all)
recommender.fit(shrink=shrink, topK=topk)
n_users_to_test = len(data_target["user_id"])
suggestions = pd.DataFrame(columns = ["user_id", "item_list"])
start_time = time.time()

for user_id in (data_target["user_id"]-1):
    suggestion, suggestion_values = recommender.recommend(user_id, cutoff=10, return_scores=True)
    suggestions.loc[len(suggestions)] = [user_id+1, " ".join([str(item_map[x]) for x in suggestion])]
    
end_time = time.time()

print("Reasonable implementation speed is {:.2f} usr/sec".format(n_users_to_test/(end_time-start_time)))

suggestions.to_csv('../SVDpp.csv', index = False)

In [1]:
# import optuna
# storage = optuna.storages.JournalStorage(
#     optuna.storages.JournalFileStorage("studies/SVDpp_study.log"),  # NFS path for distributed optimization
# )
# study = optuna.create_study(study_name = "SVDpp", storage = storage, direction="maximize")
# #study.optimize(train_evaluate, n_trials=600)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/lh/spnx4qlx6wd0sbm_1fcb_d200000gn/T/ipykernel_21704/3909308164.py:2: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  storage = optuna.storages.JournalStorage(
[I 2023-12-15 13:30:16,169] A new study created in Journal with name: SVDpp
